In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as  pd
df=pd.read_csv('/content/drive/My Drive/PRACTICE/2022SepNewsProject/Cleaned_text_7500.csv')

In [3]:
# libraries

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
np.random.seed(32)


from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Conv1D, MaxPooling1D, Dropout
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing

%matplotlib inline

In [4]:
X=df['text_lemmatized']
y=df['category']
train_text, test_text, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [5]:
Encoder = preprocessing.LabelEncoder()
y_train = Encoder.fit_transform(y_train)
y_test= Encoder.fit_transform(y_test)

In [6]:
MAX_NB_WORDS = 20000

# get the raw text data
texts_train = train_text.astype(str)
texts_test = test_text.astype(str)

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 210613 unique tokens.


In [7]:
MAX_SEQUENCE_LENGTH = 150

# pad sequences with 0s
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)

Shape of data tensor: (36750, 150)
Shape of data test tensor: (15750, 150)


In [8]:
y_train = to_categorical(np.asarray(y_train))
print('Shape of label tensor:', y_train.shape)

Shape of label tensor: (36750, 7)


In [9]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model

EMBEDDING_DIM = 50
N_CLASSES = 7

# input: a sequence of MAX_SEQUENCE_LENGTH integers
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

average = GlobalAveragePooling1D()(embedded_sequences)
predictions = Dense(N_CLASSES, activation='softmax')(average)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['acc'])

In [10]:
model.fit(x_train, y_train, validation_split=0.1,
          epochs=10, batch_size=128)

Epoch 1/10
259/259 [==============================] - 6s 21ms/step - loss: 1.7244 - acc: 0.5546 - val_loss: 1.4192 - val_acc: 0.6659
Epoch 2/10
259/259 [==============================] - 5s 19ms/step - loss: 1.1312 - acc: 0.7309 - val_loss: 0.9059 - val_acc: 0.7720
Epoch 3/10
259/259 [==============================] - 5s 20ms/step - loss: 0.7469 - acc: 0.8132 - val_loss: 0.6698 - val_acc: 0.8253
Epoch 4/10
259/259 [==============================] - 5s 19ms/step - loss: 0.5671 - acc: 0.8521 - val_loss: 0.5541 - val_acc: 0.8454
Epoch 5/10
259/259 [==============================] - 6s 22ms/step - loss: 0.4652 - acc: 0.8754 - val_loss: 0.4862 - val_acc: 0.8561
Epoch 6/10
259/259 [==============================] - 5s 19ms/step - loss: 0.3964 - acc: 0.8930 - val_loss: 0.4401 - val_acc: 0.8675
Epoch 7/10
259/259 [==============================] - 5s 19ms/step - loss: 0.3447 - acc: 0.9068 - val_loss: 0.4059 - val_acc: 0.8770
Epoch 8/10
259/259 [==============================] - 5s 19ms/step - 

In [11]:
output_test = model.predict(x_test)
#preds=model.proba(y_test)
print("test auc:", roc_auc_score(y_test,output_test,multi_class='ovr'))

test auc: 0.9864089281044605
